## --The Start--

## **IEEE30 bus system**
- Data ---->  https://github.com/MATPOWER/matpower/blob/master/data/case30.m
- 참고 ----> https://github.com/edumntg/OPF-python/blob/main/Pyomo_OPF.ipynb  (기본형)
- 침고 ----> https://github.com/yijunchengg/Optimal-Power-Flow/blob/main/optimal%20power%20flow.py (cosine 우회 방법)

In [1]:
import math
import cmath
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB

## Information
> **Bus Type**
>> ▫ 1 = PQ<br/>
>> ▫ 2 = PV<br/>
>> ▫ 3 = ref.<br/>
>> ▫ 4 = isolated<br/>

> **Generator**
>> ▫ 1, 2, 13, 22, 23, 27<br/>
>> ▫ 6 generators<br/>

> **Load**
>> ▫ 24 loads<br/>

> **IEEE bus system 해석법**


## Import Data

In [2]:
Sbase = 100
bus_df = pd.read_excel('C:/Users/user/Desktop/opf/IEEE30.xlsx', sheet_name='Bus')
bus_df.set_index('bus_i', inplace=True)
bus_df = bus_df.drop('Unnamed: 0', axis=1)

line_df = pd.read_excel('C:/Users/user/Desktop/opf/IEEE30.xlsx', sheet_name='Branch')
line_df = line_df.drop('Unnamed: 0', axis=1)

gen_df = pd.read_excel('C:/Users/user/Desktop/opf/IEEE30.xlsx', sheet_name='Gen')
gen_df = gen_df.drop('Unnamed: 0', axis=1)
gen_df.sort_values(by='bus', ascending=True, inplace=True)
gen_df.set_index('bus', inplace=True)


gencost_df = pd.read_excel('C:/Users/user/Desktop/opf/IEEE30.xlsx', sheet_name='GenCost')
gencost_df = gencost_df.drop('Unnamed: 0', axis=1)
gencost_df.sort_values(by='bus_id', ascending=True, inplace=True)
gencost_df.set_index('bus_id', inplace=True)
print(gencost_df)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/opf/IEEE30.xlsx'

In [ ]:
print(f'#===Bus Data===#')
num_bus = len(bus_df)
print(f'The number of Buses = {num_bus}')
print(f'bus_i = bus number')
print(f'Pd = real power demand')
print(f'Qd = reactive power demand')
print(f'Gs = shunt conductance')
print(f'Bs = shunt susceptance')
print(f'Vm = voltage magnitude')
print(f'Va = voltage angle')
display(bus_df.head())


print(f'#===Line Data===#')
num_line = len(line_df)
print(f'The number of Lines = {num_line}')
print(f'fbus = from bus\ntbus = to bus')
print(f'Bs = total line charging susceptance')
display(line_df.head())


print(f'#===Generator Data===#')
num_gen = len(gen_df)
print(f'The number of Generators = {num_gen}')
print(f'PG = real power output(MW)')
print(f'QG = reactive power output(MW)')
display(gen_df)

#===Bus Data===#
The number of Buses = 30
bus_i = bus number
Pd = real power demand
Qd = reactive power demand
Gs = shunt conductance
Bs = shunt susceptance
Vm = voltage magnitude
Va = voltage angle


,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
bus_i,,,,,,,,,,,,
1,3,0.0,0.0,0,0.00,1,1,0,135,1,1.05,0.95
2,2,21.7,12.7,0,0.00,1,1,0,135,1,1.10,0.95
3,1,2.4,1.2,0,0.00,1,1,0,135,1,1.05,0.95
4,1,7.6,1.6,0,0.00,1,1,0,135,1,1.05,0.95
5,1,0.0,0.0,0,0.19,1,1,0,135,1,1.05,0.95


#===Line Data===#
The number of Lines = 41
fbus = from bus
tbus = to bus
Bs = total line charging susceptance


,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.02,0.06,0.03,130,130,130,0,0,1,-360,360
1,1,3,0.05,0.19,0.02,130,130,130,0,0,1,-360,360
2,2,4,0.06,0.17,0.02,65,65,65,0,0,1,-360,360
3,3,4,0.01,0.04,0.00,130,130,130,0,0,1,-360,360
4,2,5,0.05,0.20,0.02,130,130,130,0,0,1,-360,360


#===Generator Data===#
The number of Generators = 6
PG = real power output(MW)
QG = reactive power output(MW)


,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,Pc1,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
bus,,,,,,,,,,,,,,,,,,,,
1,23.54,0,150.0,-20,1,100,1,80,0,0,0,0,0,0,0,0,0,0,0,0
2,60.97,0,60.0,-20,1,100,1,80,0,0,0,0,0,0,0,0,0,0,0,0
13,37.00,0,44.7,-15,1,100,1,40,0,0,0,0,0,0,0,0,0,0,0,0
22,21.59,0,62.5,-15,1,100,1,50,0,0,0,0,0,0,0,0,0,0,0,0
23,19.20,0,40.0,-10,1,100,1,30,0,0,0,0,0,0,0,0,0,0,0,0
27,26.91,0,48.7,-15,1,100,1,55,0,0,0,0,0,0,0,0,0,0,0,0


## List of elements

In [ ]:
slacklist = list(bus_df[bus_df['type'] == 3].index.to_numpy())
genlist = list(bus_df[(bus_df['type'] == 2) | (bus_df['type'] == 3)].index.to_numpy())
loadlist = list(bus_df[bus_df['type'] == 1].index.to_numpy())
linelist = tuple(zip(line_df['fbus'], line_df['tbus']))

print('slacklist')
print(f'the number of slack = {len(slacklist)}')
print(slacklist)
print()

print('genlist')
print(f'the number of generator = {len(genlist)}')
print(genlist)
print()

print('loadlist')
print(f'the number of load = {len(loadlist)}')
print(loadlist)

print('linelist')
print(f'the number of line = {len(linelist)}')
print(linelist)

slacklist
the number of slack = 1
[1]

genlist
the number of generator = 6
[1, 2, 13, 22, 23, 27]

loadlist
the number of load = 24
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 28, 29, 30]
linelist
the number of line = 41
((1, 2), (1, 3), (2, 4), (3, 4), (2, 5), (2, 6), (4, 6), (5, 7), (6, 7), (6, 8), (6, 9), (6, 10), (9, 11), (9, 10), (4, 12), (12, 13), (12, 14), (12, 15), (12, 16), (14, 15), (16, 17), (15, 18), (18, 19), (19, 20), (10, 20), (10, 17), (10, 21), (10, 22), (21, 22), (15, 23), (22, 24), (23, 24), (24, 25), (25, 26), (25, 27), (28, 27), (27, 29), (27, 30), (29, 30), (8, 28), (6, 28))


## Preprocessing

In [ ]:
slackset = dict()
busset = dict()
lineset = dict()
genset = dict()

slackset[1] = list(gen_df.iloc[0])

busdata = bus_df.to_numpy().astype('float64')
linedata = line_df.to_numpy().astype('float64')
gendata = gen_df.to_numpy().astype('float64')


for i, row in enumerate(busdata):
    busset[i+1] = list(row)

for i, row in enumerate(linedata):
    lineset[i+1] = list(row)

for row in gendata:
    genset[row[0]] = list(row)

- 번호가 0부터 매겨져 있기 때문에 바꿔줌.
- 데이터 접근 높이기 위해 row 단위로 끊어줌.

## Create an admittance(Y) matrix

In [ ]:
Ybus = np.zeros((num_bus, num_bus), dtype=np.complex128)
g = np.zeros((num_bus, num_bus))
b = np.zeros((num_bus, num_bus))

# 순서대로 list 하나씩 뽑아오는 거임.
for lineid, lineinfo in lineset.items():

    i = int(lineinfo[0]) - 1  # from # Y bus는 (0, 0)부터 시작하니까 우리 데이터에서 -1 해줌.
    k = int(lineinfo[1]) - 1  # to

    z = complex(lineinfo[2], lineinfo[3])
    Bs = complex(0, lineinfo[4])
    #a = lineinfo[5] # tap인데 여기서는 안 씀.

    # lineinfo[0]랑 lineinfo[1]는 연결 상태를 말해줌.
    # 그래서 lineinfo[0]는 1, 1, 2, 3, 2, 2,.... 순서로 나옴.
    # diagonal term은 연결된 Y들의 총합이므로   +=   로 표현해준 거임.
    Ybus[i, i] += 1/z # from
    Ybus[k, k] += 1/z # 중복 line 없기 때문에 to에 해당하는 node도 이렇게 해줌.

    Ybus[i][i] += Bs
    Ybus[k][k] += Bs

    Ybus[i][k] -= 1/z
    Ybus[k][i] -= 1/z

    b[i][k] = Bs.imag
    b[k][i] = Bs.imag

    G = Ybus.real
    B = Ybus.imag

#print(Ybus)
print(G.shape)
#print(G)
#print(B)

(30, 30)


\begin{equation}
\begin{aligned}

& \textrm{AC Optimal Power Flow Formulation}\\
\min_{g} \quad & \sum_{\mathcal{g} \in \mathcal{G}} F(P_{g}) \\
\textrm{s.t.} \quad & \theta_{ref.} = 0, \quad ref. \in \mathcal{B}\\
                    & P_{g}^{min} \le P_{g} \le P_{g}^{max}, \quad \forall \mathcal{g} \in \mathcal{G}\\
                    & Q_{g}^{min} \le Q_{g} \le Q_{g}^{max}, \quad \forall \mathcal{g} \in \mathcal{G}\\
                    & V_{b}^{min} \le V_{b} \le V_{b}^{max}, \quad \forall \mathcal{b} \in \mathcal{B}\\
                    & -\theta_{ij}^{\varDelta} \le \angle (V_iV_j^*) \le \theta_{ij}^{\varDelta}, \quad \forall (i,j) \in \mathcal{E}\\
                    & P_{ij}^2 + Q_{ij}^2 \le S_{ij}^2 \quad \forall (i,j) \in \mathcal{E}\\
                    & (P_{g} - P_{d}) - \sum_{}

\end{aligned}
\end{equation}

## Create a new model

In [ ]:
m = gp.Model('ACOPF')
m.params.NonConvex = 2

Set parameter NonConvex to value 2


## Create new variables

In [ ]:
Pg = dict()
Qg = dict()

V_mag = dict()
V_ang = dict()

Pflow = dict()
Qflow = dict()

# for Generators.
for num in np.arange(1, 31):

    if num in genlist:
        Pg[num] = m.addVar(lb=gen_df['Pmin'][num], ub=gen_df['Pmax'][num], name=(f'Pg{num}') )
        Qg[num] = m.addVar(lb=gen_df['Qmin'][num], ub=gen_df['Qmax'][num], name=(f'Qg{num}') )

    else: # Generator 없는 bus들은 모두 0으로 지정
        Pg[num] = m.addVar(lb=0, ub=0, name=(f'P_{num}_gen') )
        Qg[num] = m.addVar(lb=0, ub=0, name=(f'Q_{num}_gen') )


# for bus voltages
for num in np.arange(1, 31):

    if num in slacklist: # slack의 angle은 0으로 지정
        V_mag[num] = m.addVar(lb=bus_df['Vmin'][num], ub=bus_df['Vmax'][num], name=(f'V_mag{num}'))
        V_ang[num] = m.addVar(lb=0, ub=0, name=f'V_ang{num}')

    else:
        V_mag[num] = m.addVar(lb=bus_df['Vmin'][num], ub=bus_df['Vmax'][num], name=(f'V_mag{num}'))
        V_ang[num] = m.addVar(lb=-2*np.pi, ub=2*np.pi, name=f'V_ang{num}')

# for branches
for i, j in linelist:

    Pflow[i, j] = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name=f'Pflow({i},{j})')
    Qflow[i, j] = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name=f'Qflow({i},{j})')


m.update() # 이거 해줘야 variable이 제대로 반영됨.

## Create some additional variables to support the problem related to _cosine_

In [ ]:
Vi_Vn = dict()
theta_in = dict()
theta_sum = dict()
cos_theta = dict()
sin_theta = dict()

# 세 개 이상의 변수를 곱하면 Gurobi가 동작 안 하기 때문에 만들어줌.
for i in np.arange(1, 31):
    for n in np.arange(1, 31):
        Vi_Vn[i,n] = m.addVar(lb=bus_df['Vmin'][num]*bus_df['Vmin'][num],\
                                ub=bus_df['Vmax'][num]*bus_df['Vmax'][num], name=f'V{i}*V{n}')

        theta_sum[i,n] = m.addVar(lb=-np.pi, ub=np.pi, name=f'theta_sum({i},{n})')

        # 
        theta_in[i,n] = m.addVar(lb=-2*np.pi, ub=+2*np.pi, name=f'theta_({i},{n})')
        

        # np.cos( )에 변수를 argument로 넣을 수 없기 때문에 만들어줌.
        cos_theta[i,n] = m.addVar(lb=-1, ub=+1, name=f'cos of theta({i},{n})')
        sin_theta[i,n] = m.addVar(lb=-1, ub=+1, name=f'sin of theta({i},{n})')


m.update() # 이거 실행해야 variable이 제대로 적용됨.

# Supportive constraints due to the limitations of Gurobi solver
# constratins 형태로 넣었지만, 걍 단순히 성립 관계를 설명해 주는 거임.
for i, n in linelist:

    m.addConstr(Vi_Vn[i,n] == V_mag[i]*V_mag[n])
    m.addConstr(theta_in[i,n] == V_ang[i] - V_ang[n])
    m.addConstr(theta_sum[i,n] == theta_in[i,n] - np.abs(Ybus[i-1, n-1]))

    m.addGenConstrCos(theta_sum[i,n], cos_theta[i,n])
    m.addGenConstrSin(theta_sum[i,n], sin_theta[i,n])

## Usual constraints
- box constraints는 대부분 variable 만들면서 정의했음.

- power balance equations


In [ ]:
print(type(Ybus))

<class 'numpy.ndarray'>


In [ ]:
# gp.quicksum( ) 있어야 for까지 한번에 묶을 수 있음.

for i in np.arange(1, 31):
    m.addConstr(Pg[i] - bus_df['Pd'][i]  ==  gp.quicksum(Vi_Vn[i,n]*np.abs(Ybus[i-1, n-1])*\
                                            cos_theta[i,n]  for n in np.arange(1, 31))   )

- line flow limit

In [ ]:
for i, j in linelist:

    cond = (line_df['fbus'] == i) & (line_df['tbus'] == j)
    m.addConstr(Pflow[i,j]*Pflow[i,j] + Qflow[i,j]*Qflow[i,j]  <=  line_df['rateA'][cond])

m.update()

## Set an objective function

In [ ]:
m.setObjective(gp.quicksum(gencost_df['c2'][i]*Pg[i]**2 + gencost_df['c1'][i]*Pg[i] + gencost_df['c0'][i] for i in genlist), GRB.MINIMIZE)

## Optimize the problem

In [ ]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 82 rows, 4702 columns and 205 nonzeros
Model fingerprint: 0x71241998
Model has 6 quadratic objective terms
Model has 112 quadratic constraints
Model has 82 general constraints
Variable types: 4702 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 9e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  QObjective range [2e-02, 1e-01]
  Bounds range     [9e-01, 2e+02]
  RHS range        [1e+00, 4e+01]
  QRHS range       [2e+00, 1e+02]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


## --The End--